In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

In [3]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)

In [5]:
import pathlib
data_dir = pathlib.Path(data_dir)

In [6]:
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

In [7]:
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [8]:
X = []
y = []

In [9]:
for flower_name , images in flowers_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        reshaped = cv2.resize(img , (224,224))
        X.append(reshaped)
        y.append(flowers_labels_dict[flower_name])

In [10]:
X = np.array(X)
y = np.array(y)

In [11]:
X = X/255

In [12]:
from sklearn.model_selection import train_test_split
X_train , X_test ,  y_train , y_test = train_test_split(X , y  , test_size =0.33)

In [13]:
X_train.shape

(2458, 224, 224, 3)

In [79]:
import gc
gc.collect()

2080

In [22]:
m = tf.keras.Sequential([
   hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4", output_shape=[5])
])
m.build([None, 224, 224, 3]) # Batch input shape.
# cant use for tranfer learning because it consist last layer


In [14]:
pretrained_model_without_top = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", input_shape = (224,224,3),
trainable=False)

In [15]:
model = keras.Sequential([
    pretrained_model_without_top,
    keras.layers.Dense(5)
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [21]:
model2 = keras.Sequential([
    pretrained_model_without_top,
    keras.layers.Dense(5)
])
model2.compile(optimizer = "adam" , loss = "sparse_categorical_crossentropy" , metrics = ["accuracy"])
model2.fit(X_train, y_train , epochs = 15)

Epoch 1/15
77/77 [==============================] - 10s 132ms/step - loss: 1.9601 - accuracy: 0.5358
Epoch 2/15
77/77 [==============================] - 10s 133ms/step - loss: 1.4920 - accuracy: 0.6371
Epoch 3/15
77/77 [==============================] - 10s 133ms/step - loss: 1.3250 - accuracy: 0.5822
Epoch 4/15
77/77 [==============================] - 10s 133ms/step - loss: 1.1836 - accuracy: 0.7189
Epoch 5/15
77/77 [==============================] - 10s 132ms/step - loss: 1.2026 - accuracy: 0.7762
Epoch 6/15
77/77 [==============================] - 10s 132ms/step - loss: 1.4013 - accuracy: 0.7762
Epoch 7/15
77/77 [==============================] - 10s 131ms/step - loss: 1.9386 - accuracy: 0.6497
Epoch 8/15
77/77 [==============================] - 10s 131ms/step - loss: 1.5327 - accuracy: 0.7148
Epoch 9/15
77/77 [==============================] - 10s 130ms/step - loss: 1.5640 - accuracy: 0.7465
Epoch 10/15
77/77 [==============================] - 10s 131ms/step - loss: 1.5675 - accura

In [24]:
model2.evaluate(X_test, y_test)

38/38 [==============================] - 5s 132ms/step - loss: 1.5483 - accuracy: 0.7195


[1.5483424663543701, 0.7194719314575195]

In [26]:
y_predict = model2.predict(X_test)

In [30]:
np.argmax(y_predict[1])

0

In [34]:
sample = model2.predict(np.array([X[-1]]))

In [35]:
np.argmax(sample)

3